In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.metrics import precision_recall_curve
from collections import Counter
import numpy as np
import json
import re
import os
import math

# -------------------
# Utility Functions
# -------------------
def check_gpu_availability():
    """Check if CUDA is available and print GPU details if present."""
    is_cuda_available = torch.cuda.is_available()
    print("CUDA available:", is_cuda_available)
    if is_cuda_available:
        print("GPU device:", torch.cuda.get_device_name(0))
        print("GPU memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")
    return torch.device("cuda:0" if is_cuda_available else "cpu")

def diplomacy_tokenizer(text):
    """Tokenize text into words and punctuation, converting to lowercase.
    
    Args:
        text (str): Input text to tokenize.
    
    Returns:
        list: List of tokens (words and punctuation).
    """
    text = text.lower()
    return re.findall(r"\w+|[.,!?;]", text)

def load_jsonl(file_path):
    """Load data from a JSONL file.
    
    Args:
        file_path (str): Path to the JSONL file.
    
    Returns:
        list: List of dictionaries parsed from JSONL.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

# -------------------
# Vocabulary Class
# -------------------
class Vocab:
    def __init__(self, min_freq=2):
        """Initialize vocabulary with special tokens and minimum frequency.
        
        Args:
            min_freq (int): Minimum frequency for tokens to be included in vocab.
        """
        self.token_to_idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx_to_token = ['<PAD>', '<UNK>']
        self.min_freq = min_freq
        self.harbinger_indices = set()

    def build_vocab(self, texts, harbingers):
        """Build vocabulary from texts, including harbinger tokens.
        
        Args:
            texts (list): List of text messages to build vocab from.
            harbingers (list): List of harbinger tokens to track.
        """
        counter = Counter()
        for text in texts:
            counter.update(diplomacy_tokenizer(text))
        for token, freq in counter.items():
            if freq >= self.min_freq:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
        self.harbinger_indices = set(self.token_to_idx.get(token, 1) for token in harbingers
                                    if token in self.token_to_idx)

    def encode(self, tokens):
        """Encode a list of tokens into indices.
        
        Args:
            tokens (list): List of tokens to encode.
        
        Returns:
            list: List of token indices, using <UNK> for unknown tokens.
        """
        return [self.token_to_idx.get(t, 1) for t in tokens]

    def __len__(self):
        """Return the size of the vocabulary."""
        return len(self.idx_to_token)

# -------------------
# Dataset Class
# -------------------
class DiplomacyDataset(Dataset):
    def __init__(self, messages, labels, vocab, context_size=2, max_len=300):
        """Initialize dataset for Diplomacy messages.
        
        Args:
            messages (list): List of text messages.
            labels (list): List of binary labels (0=truthful, 1=deceptive).
            vocab (Vocab): Vocabulary object for token encoding.
            context_size (int): Number of previous messages to include as context.
            max_len (int): Maximum length of token sequence.
        """
        self.messages = messages
        self.labels = labels
        self.vocab = vocab
        self.context_size = context_size
        self.max_len = max_len
        self.encoded = [vocab.encode(diplomacy_tokenizer(msg)) for msg in messages]

    def __len__(self):
        """Return the number of samples in the dataset."""
        return len(self.messages)

    def __getitem__(self, idx):
        """Get a sample with context, harbinger mask, and label.
        
        Args:
            idx (int): Index of the sample.
        
        Returns:
            tuple: (context_tensor, harbinger_mask_tensor, label_tensor)
        """
        context = []
        for i in range(max(0, idx - self.context_size), idx + 1):
            context.extend(self.encoded[i])
        if len(context) < self.max_len:
            context += [0] * (self.max_len - len(context))
        else:
            context = context[:self.max_len]
        harbinger_mask = [1 if token in self.vocab.harbinger_indices else 0 for token in context]
        context_tensor = torch.tensor(context, dtype=torch.long)
        harbinger_mask_tensor = torch.tensor(harbinger_mask, dtype=torch.float)
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.float)
        return context_tensor, harbinger_mask_tensor, label_tensor

# -------------------
# Model Components
# -------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        """Initialize positional encoding for transformer.
        
        Args:
            d_model (int): Dimension of the model (embedding + features).
            max_len (int): Maximum sequence length.
        """
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model % 2 == 0:
            pe[:, 1::2] = torch.cos(position * div_term)
        else:
            pe[:, 1::2] = torch.cos(position * div_term[:-1])
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """Add positional encoding to input embeddings.
        
        Args:
            x (tensor): Input tensor of shape (batch_size, seq_len, d_model).
        
        Returns:
            tensor: Input with positional encoding added.
        """
        return x + self.pe[:, :x.size(1), :]

class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=127, num_heads=4, num_layers=2, hidden_dim=256, dropout=0.3, max_len=300):
        """Initialize transformer-based classifier.
        
        Args:
            vocab_size (int): Size of the vocabulary.
            embed_dim (int): Embedding dimension.
            num_heads (int): Number of attention heads.
            num_layers (int): Number of transformer encoder layers.
            hidden_dim (int): Dimension of feedforward network.
            dropout (float): Dropout rate.
            max_len (int): Maximum sequence length.
        """
        super().__init__()
        assert (embed_dim + 1) % num_heads == 0, "embed_dim + 1 must be divisible by num_heads"
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        d_model = embed_dim + 1
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=hidden_dim,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x, harb_mask):
        """Forward pass through the model.
        
        Args:
            x (tensor): Input token indices (batch_size, seq_len).
            harb_mask (tensor): Harbinger mask (batch_size, seq_len).
        
        Returns:
            tensor: Logits (batch_size,).
        """
        embedded = self.embedding(x)
        harb_mask = harb_mask.unsqueeze(2)
        combined = torch.cat([embedded, harb_mask], dim=2)
        combined = self.pos_encoder(combined)
        mask = (x == 0)
        output = self.transformer_encoder(combined, src_key_padding_mask=mask)
        output = output.mean(dim=1)
        output = self.dropout(output)
        return self.fc(output).squeeze(1)

# -------------------
# Training and Evaluation
# -------------------
def train_epoch(model, dataloader, optimizer, criterion, device):
    """Train the model for one epoch.
    
    Args:
        model (nn.Module): The model to train.
        dataloader (DataLoader): Training data loader.
        optimizer (optim.Optimizer): Optimizer.
        criterion (nn.Module): Loss function.
        device (torch.device): Device to run computations on.
    
    Returns:
        float: Average loss for the epoch.
    """
    model.train()
    losses = []
    for x, harb_mask, y in dataloader:
        x, harb_mask, y = x.to(device), harb_mask.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x, harb_mask)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

def evaluate(model, dataloader, device, criterion=None):
    """Evaluate the model on a dataset.
    
    Args:
        model (nn.Module): The model to evaluate.
        dataloader (DataLoader): Data loader for evaluation.
        device (torch.device): Device to run computations on.
        criterion (nn.Module, optional): Loss function to compute validation loss.
    
    Returns:
        tuple: Arrays of logits, true labels, and average loss (if criterion provided).
    """
    model.eval()
    all_logits, all_labels, losses = [], [], []
    with torch.no_grad():
        for x, harb_mask, y in dataloader:
            x, harb_mask, y = x.to(device), harb_mask.to(device), y.to(device)
            logits = model(x, harb_mask)
            all_logits.extend(logits.cpu().tolist())
            all_labels.extend(y.cpu().tolist())
            if criterion:
                loss = criterion(logits, y)
                losses.append(loss.item())
    return np.array(all_logits), np.array(all_labels), np.mean(losses) if losses else None

def print_metrics(true, preds, probs, name=""):
    """Print classification metrics.
    
    Args:
        true (array): True labels.
        preds (array): Predicted labels.
        probs (array): Predicted probabilities.
        name (str): Dataset name for display.
    """
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds)
    auc = roc_auc_score(true, probs)
    print(f"\n{name} Set Performance")
    print(f"Accuracy: {acc:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")
    print(classification_report(true, preds))

def find_best_threshold(logits, labels):
    """Find the optimal classification threshold based on F1 score.
    
    Args:
        logits (array): Model logits.
        labels (array): True labels.
    
    Returns:
        tuple: Best threshold and corresponding F1 score.
    """
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    precision, recall, thresholds = precision_recall_curve(labels, probs)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
    best_idx = np.argmax(f1_scores)
    return thresholds[best_idx], f1_scores[best_idx]

def track_losses(train_loss, val_loss, epoch, train_losses, val_losses):
    """Track and log training and validation losses.
    
    Args:
        train_loss (float): Training loss for the current epoch.
        val_loss (float): Validation loss for the current epoch.
        epoch (int): Current epoch number.
        train_losses (list): List to store training losses.
        val_losses (list): List to store validation losses.
    
    Returns:
        tuple: Updated train_losses and val_losses lists.
    """
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    print(f"Epoch {epoch+1} Losses - Train: {train_loss:.4f}, Validation: {val_loss:.4f}")
    return train_losses, val_losses

# -------------------
# Data Preprocessing
# -------------------
def preprocess_messages(data):
    """Extract messages and labels from game data.
    
    Args:
        data (list): List of game dictionaries with messages and labels.
    
    Returns:
        tuple: Messages, labels, count of truthful, count of deceptive.
    """
    messages, labels = [], []
    for game in data:
        game_messages = game.get("messages", [])
        sender_labels = game.get("sender_labels", [])
        if not game_messages or len(game_messages) < 2 or len(sender_labels) != len(game_messages):
            continue
        for i, msg in enumerate(game_messages):
            is_deceptive = 0 if sender_labels[i] else 1
            messages.append(msg)
            labels.append(is_deceptive)
    num_deceptive = sum(labels)
    num_truthful = len(labels) - num_deceptive
    print(f"Processed {len(messages)} messages with {num_deceptive} deceptive and {num_truthful} truthful")
    return messages, labels, num_truthful, num_deceptive

def load_datasets(data_dir):
    """Load training, validation, and test datasets.
    
    Args:
        data_dir (str): Directory containing JSONL files.
    
    Returns:
        tuple: Train, validation, and test data.
    """
    train_data = load_jsonl("/kaggle/input/deception/train.jsonl")
    val_data = load_jsonl("/kaggle/input/deception/validation.jsonl")
    test_data = load_jsonl("/kaggle/input/deception/test.jsonl")
    print(f"Loaded {len(train_data)} training games")
    return train_data, val_data, test_data

# -------------------
# Training Pipeline
# -------------------
def setup_training_components(train_msgs, train_labels, val_msgs, val_labels, test_msgs, test_labels, harbingers, config):
    """Set up vocabulary, datasets, data loaders, and model.
    
    Args:
        train_msgs (list): Training messages.
        train_labels (list): Training labels.
        val_msgs (list): Validation messages.
        val_labels (list): Validation labels.
        test_msgs (list): Test messages.
        test_labels (list): Test labels.
        harbingers (list): Harbinger tokens.
        config (dict): Configuration dictionary.
    
    Returns:
        tuple: Model, optimizer, criterion, data loaders, vocabulary.
    """
    vocab = Vocab(min_freq=2)
    vocab.build_vocab(train_msgs, harbingers)
    print(f"Vocabulary size: {len(vocab)}")

    train_num_truthful = len(train_labels) - sum(train_labels)
    train_num_deceptive = sum(train_labels)
    pos_weight = train_num_truthful / train_num_deceptive if train_num_deceptive > 0 else 1.0
    print(f"Positive weight (truthful/deceptive): {pos_weight:.4f}")

    train_set = DiplomacyDataset(train_msgs, train_labels, vocab, config['context_size'], config['max_len'])
    val_set = DiplomacyDataset(val_msgs, val_labels, vocab, config['context_size'], config['max_len'])
    test_set = DiplomacyDataset(test_msgs, test_labels, vocab, config['context_size'], config['max_len'])

    train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_set, batch_size=config['batch_size'], num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_set, batch_size=config['batch_size'], num_workers=2, pin_memory=True)

    model = TransformerClassifier(
        vocab_size=len(vocab),
        embed_dim=config['embed_dim'],
        num_heads=config['num_heads'],
        num_layers=config['num_layers'],
        hidden_dim=config['hidden_dim'],
        dropout=config['dropout'],
        max_len=config['max_len']
    ).to(config['device'])

    optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight, dtype=torch.float).to(config['device']))

    return model, optimizer, criterion, train_loader, val_loader, test_loader, vocab

def run_training():
    """Main training function orchestrating the training pipeline."""
    config = {
        'data_dir': "/kaggle/input/diplomacy" if os.path.exists("/kaggle/input") else ".",
        'batch_size': 32,
        'context_size': 2,
        'max_len': 300,
        'embed_dim': 127,
        'num_heads': 4,
        'num_layers': 2,
        'hidden_dim': 256,
        'dropout': 0.3,
        'lr': 1e-4,
        'num_epochs': 15,
        'early_stop_patience': 3,
        'device': check_gpu_availability()
    }

    harbingers = [
        "maybe", "perhaps", "possibly", "might", "could", "would", "should", "potentially", "presumably",
        "likely", "unlikely", "probably", "possibly", "conceivably", "hopefully", "eventually", "ultimately",
        "definitely", "certainly", "surely", "absolutely", "undoubtedly", "clearly", "obviously", "apparently",
        "seemingly", "allegedly", "supposedly", "reportedly", "essentially", "basically", "fundamentally",
        "significantly", "considerably", "virtually", "practically", "nearly", "almost", "marginally", "somewhat",
        "think", "believe", "feel", "suppose", "guess", "wonder", "assume", "suspect", "estimate", "imagine",
        "figure", "reckon", "expect", "predict", "anticipate", "foresee", "presume", "infer", "deduce", "conclude",
        "gather", "surmise", "speculate", "theorize", "hypothesize", "sense", "perceive", "notice", "realize",
        "recognize", "understand", "know", "remember", "recall", "forget",
        "sort", "kind", "rather", "quite", "somewhat", "slightly", "moderately", "relatively", "comparatively",
        "reasonably", "fairly", "pretty", "mostly", "mainly", "primarily", "partially", "largely", "substantially",
        "typically", "generally", "usually", "normally", "commonly", "regularly", "often", "frequently", "sometimes",
        "occasionally", "rarely", "seldom",
        "very", "really", "extremely", "absolutely", "totally", "completely", "utterly", "perfectly", "entirely",
        "thoroughly", "fully", "wholly", "downright", "positively", "simply", "just", "merely", "only", "literally",
        "actually", "honestly", "truly", "genuinely", "sincerely", "frankly",
        "about", "around", "approximately", "roughly", "nearly", "close to", "in the range of", "something like",
        "or so", "more or less", "give or take", "in general", "on the whole", "all things considered", "by and large",
        "for the most part", "to some extent", "in some ways", "in a sense", "in theory", "technically",
        "strictly speaking", "officially", "formally", "nominally", "effectively", "in effect", "in principle",
        "ideally", "theoretically",
        "now", "currently", "presently", "at present", "at the moment", "these days", "lately", "recently",
        "in recent times", "over time", "with time", "in time", "sooner or later", "eventually", "ultimately",
        "in the end", "at the end of the day", "when all is said and done", "in the long run", "in the final analysis",
        "according to", "as per", "based on", "in light of", "in view of", "given that", "seeing as", "considering",
        "taking into account", "from what I understand", "from what I gather", "from my perspective", "in my opinion",
        "to my knowledge", "as far as I know",
        "diplomatically", "strategically", "tactically", "politically", "negotiable", "flexible", "adaptable",
        "revisable", "amendable", "subject to", "conditional upon", "dependent on", "contingent on", "pending",
        "awaiting", "considering", "reviewing", "evaluating", "assessing", "monitoring", "observing", "watching",
        "tracking", "following", "pursuant to"
    ]

    train_data, val_data, test_data = load_datasets(config['data_dir'])
    train_msgs, train_labels, _, _ = preprocess_messages(train_data)
    val_msgs, val_labels, _, _ = preprocess_messages(val_data)
    test_msgs, test_labels, _, _ = preprocess_messages(test_data)

    model, optimizer, criterion, train_loader, val_loader, test_loader, vocab = setup_training_components(
        train_msgs, train_labels, val_msgs, val_labels, test_msgs, test_labels, harbingers, config
    )

    best_val_f1 = 0
    best_threshold = 0.5
    epochs_without_improvement = 0
    best_model_path = os.path.join("/kaggle/working" if os.path.exists("/kaggle/working") else ".", "best_diplomacy_model.pth")
    train_losses, val_losses = [], []

    for epoch in range(config['num_epochs']):
        print(f"\nStarting Epoch {epoch+1}...")
        train_loss = train_epoch(model, train_loader, optimizer, criterion, config['device'])

        val_logits, val_true, val_loss = evaluate(model, val_loader, config['device'], criterion)
        train_losses, val_losses = track_losses(train_loss, val_loss, epoch, train_losses, val_losses)

        val_probs = torch.sigmoid(torch.tensor(val_logits)).numpy()
        val_preds = (val_probs > 0.5).astype(int)
        val_f1 = f1_score(val_true, val_preds)
        print(f"Validation F1 (threshold=0.5): {val_f1:.4f}")

        current_threshold, current_f1 = find_best_threshold(val_logits, val_true)
        print(f"Best threshold: {current_threshold:.4f}, Best F1: {current_f1:.4f}")

        if current_f1 > best_val_f1:
            best_val_f1 = current_f1
            best_threshold = current_threshold
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with F1: {current_f1:.4f}")
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            print(f"No improvement for {epochs_without_improvement} epoch(s).")
            if epochs_without_improvement >= config['early_stop_patience']:
                print(f"Early stopping triggered after {config['early_stop_patience']} epochs.")
                break

    model.load_state_dict(torch.load(best_model_path))
    for name, loader, labels in [("Validation", val_loader, val_labels), ("Test", test_loader, test_labels)]:
        logits, true, _ = evaluate(model, loader, config['device'])
        probs = torch.sigmoid(torch.tensor(logits)).numpy()
        preds = (probs > best_threshold).astype(int)
        print_metrics(true, preds, probs, name)

if __name__ == "__main__":
    run_training()

CUDA available: True
GPU device: Tesla P100-PCIE-16GB
GPU memory: 17.059545088 GB
Loaded 189 training games
Processed 13128 messages with 590 deceptive and 12538 truthful
Processed 1416 messages with 56 deceptive and 1360 truthful
Processed 2741 messages with 240 deceptive and 2501 truthful
Vocabulary size: 4650
Positive weight (truthful/deceptive): 21.2508

Starting Epoch 1...
Epoch 1 Losses - Train: 1.3315, Validation: 1.2904
Validation F1 (threshold=0.5): 0.1083
Best threshold: 0.5112, Best F1: 0.1220
New best model saved with F1: 0.1220

Starting Epoch 2...
Epoch 2 Losses - Train: 1.3089, Validation: 1.2885
Validation F1 (threshold=0.5): 0.1159
Best threshold: 0.5021, Best F1: 0.1245
New best model saved with F1: 0.1245

Starting Epoch 3...
Epoch 3 Losses - Train: 1.2763, Validation: 1.2829
Validation F1 (threshold=0.5): 0.0956
Best threshold: 0.5311, Best F1: 0.1296
New best model saved with F1: 0.1296

Starting Epoch 4...
Epoch 4 Losses - Train: 1.2673, Validation: 1.2814
Validat

/tmp/ipykernel_31/34449727.py:518: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))



Validation Set Performance
Accuracy: 0.8475, F1: 0.1220, AUC: 0.6253
              precision    recall  f1-score   support

         0.0       0.97      0.87      0.92      1360
         1.0       0.08      0.27      0.12        56

    accuracy                           0.85      1416
   macro avg       0.52      0.57      0.52      1416
weighted avg       0.93      0.85      0.89      1416


Test Set Performance
Accuracy: 0.8121, F1: 0.2161, AUC: 0.6395
              precision    recall  f1-score   support

         0.0       0.93      0.86      0.89      2501
         1.0       0.17      0.30      0.22       240

    accuracy                           0.81      2741
   macro avg       0.55      0.58      0.55      2741
weighted avg       0.86      0.81      0.83      2741

